## Data partitioning

In this notebook we will take the conditioned data that we have, the data should be in the form of a two column csv that has the block numvber on the left and the MEV quantity sorted in to none = 0, low = 1, medium = 2, and high =3. This notebook is specifically for the data with the gas fees and the price volatility of etherium at the time. 

In [35]:
import numpy as np 
import pandas as pd
from numpy import genfromtxt
import multiprocessing as mp
from multiprocessing import Pool
from numba import jit
import json
import csv

In [36]:

#let pull the data in and so that we can check how big the files are. 

my_data = genfromtxt('..\Ethdata\MEVcategorywgv.csv', delimiter=',')
print(my_data)
print(np.shape(my_data))
my_data = np.delete(my_data, 0,1) #just to get rid of the index row
my_data = np.delete(my_data, 0,1) #just to get rid of the index row
my_data = np.delete(my_data, 0,1) #just to get rid of the index row
my_data = np.delete(my_data,range(0,50),0) #and the nan values at the top

print(my_data)
print(np.shape(my_data))


[[           nan            nan            nan            nan
             nan            nan]
 [0.00000000e+00 0.00000000e+00 1.18340490e+07 4.00000000e+00
  9.99796494e-01            nan]
 [1.00000000e+00 1.00000000e+00 1.18340500e+07 0.00000000e+00
  9.99850057e-01            nan]
 ...
 [5.20300000e+03 5.20300000e+03 1.18392520e+07 0.00000000e+00
  9.96924509e-01 3.14770091e+08]
 [5.20400000e+03 5.20400000e+03 1.18392530e+07 0.00000000e+00
  9.99256346e-01 3.05603242e+08]
 [5.20500000e+03 5.20500000e+03 1.18392540e+07 0.00000000e+00
  9.99197509e-01 2.95728027e+08]]
(5207, 6)
[[0.00000000e+00 9.87959922e-01 1.51566715e+07]
 [0.00000000e+00 9.87496250e-01 1.50104750e+07]
 [0.00000000e+00 9.99704350e-01 1.48502837e+07]
 ...
 [0.00000000e+00 9.96924509e-01 3.14770091e+08]
 [0.00000000e+00 9.99256346e-01 3.05603242e+08]
 [0.00000000e+00 9.99197509e-01 2.95728027e+08]]
(5157, 3)


In [37]:
# This cell will make what I am refering to as datalets, these are strings of n periods, infact this will create all of the strings of n periods from the data available
# we will use these to train tha models. 

#this parameter will set what the maximum datalet length will be i.e. how many blocks back we want to go to predict the MEV level of the next block

startrange = 2

endrange = 10
#start by making a dictionary to hold all this data

datalets = {}

# Then we loop over, the default is set from 2-10, this is the length of hte data length, the x vector will be one less than this. 

for i in range(startrange ,endrange):
    
    # Start with an all zeros np array so we can get started, it will have 3i collumns as each period has three bits of data, the mev, gas fees and price volitility
    
    datalet = np.zeros((np.shape(my_data)[0],i*3))
    
    # these lines wil shift the rows up and add them as the next collumn in the datalets, it is done three times to do it for the MEV, gas fees and price volatility
    
    for j in range(i):
        data = np.delete(my_data[:,0],range(j),0)
        data = np.append(data,range(j))
        data = data.astype(int)
        datalet[:,j] = data

        data = np.delete(my_data[:,1],range(j),0)
        data = np.append(data,range(j))
        data = data.astype(float)
        datalet[:,i+j] = data        
        
        data = np.delete(my_data[:,2],range(j),0)
        data = np.append(data,range(j))
        data = data.astype(float)
        datalet[:,2*i+j] = data        
        
    #in the process above we create some nonsense rows and this just makes sure they are removed
               
    datalet = datalet.astype(float)
    datalet = np.delete(datalet,range(i*(-1),0),0)
        
    datalets["datalet{}".format(i)] = datalet

print(datalets)  

{'datalet2': array([[0.00000000e+00, 0.00000000e+00, 9.87959922e-01, 9.87496250e-01,
        1.51566715e+07, 1.50104750e+07],
       [0.00000000e+00, 0.00000000e+00, 9.87496250e-01, 9.99704350e-01,
        1.50104750e+07, 1.48502837e+07],
       [0.00000000e+00, 0.00000000e+00, 9.99704350e-01, 9.98955092e-01,
        1.48502837e+07, 1.46756392e+07],
       ...,
       [0.00000000e+00, 0.00000000e+00, 9.98571011e-01, 9.96847309e-01,
        3.31220993e+08, 3.23288838e+08],
       [0.00000000e+00, 0.00000000e+00, 9.96847309e-01, 9.96924509e-01,
        3.23288838e+08, 3.14770091e+08],
       [0.00000000e+00, 0.00000000e+00, 9.96924509e-01, 9.99256346e-01,
        3.14770091e+08, 3.05603242e+08]]), 'datalet3': array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.51566715e+07, 1.50104750e+07, 1.48502837e+07],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.50104750e+07, 1.48502837e+07, 1.46756392e+07],
       [0.00000000e+00, 0.00000000e+00, 0.00000

## Saving these datalets as CSVs

In [38]:
#first we just need to make the column headings which is what the headers do, then save it all to a csv file. 

for x,y in datalets.items():
    headers = []
    for i in range(int(np.shape(y)[1]/3)):
        headers.append('Mev_period{}'.format(i))
    for i in range(int(np.shape(y)[1]/3)):
        headers.append('gas_fees_period{}'.format(i))
    for i in range(int(np.shape(y)[1]/3)):
        headers.append('price_volitility_period{}'.format(i))
    headerstr = ','.join(map(str,headers))
    np.savetxt("..\dataletswgv\{}.csv".format(x), y, delimiter=",", header= headerstr, comments = '')